In [ ]:
!pip install transformers==4.3.3
!pip install numpy==1.20.1
!pip install SoundFile==0.10.3.post1
!pip install torch


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')



Mounted at /content/gdrive


In [4]:
import sys
!unzip -q "/content/gdrive/MyDrive/final_dataset.zip" 
!ls /content/gdrive/My\ Drive/*.py
sys.path.append('/content/gdrive/My Drive')


'/content/gdrive/My Drive/data.py'  '/content/gdrive/My Drive/model.py'


In [5]:
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader
from data import Data
import numpy as np
from model import Wav2Vec2
import json
import os
dataset = []
label = []
if not os.path.exists("data.json"):
    data_path = "/content/Final"
    class_path = os.listdir(data_path)
    for ind, i in enumerate(class_path):
        temp_path = []
        temp_label = []
        for j in os.listdir(os.path.join(data_path,i)):
            temp_path.append(os.path.join(data_path,i,j))
            label.append(ind)
        dataset.extend(temp_path)
        label.extend(temp_label)
    with open('data.json','w') as f:
        json.dump({'data':dataset,
                'label':label},f)   
else:
    with open('data.json','r') as f:
        temp = json.load(f) 
        dataset = temp['data']
        label = temp['label']

In [7]:
from torch.utils.data.sampler import SubsetRandomSampler
data = Data(dataset, label)
#data = DataLoader(data,batch_size=batch, shuffle=True)
batch_size = 64
validation_split = .2
shuffle_dataset = True
random_seed= 42

# Creating data indices for training and validation splits:
dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_dataloader =  torch.utils.data.DataLoader(data, batch_size=batch_size,sampler=train_sampler)
test_dataloader =  torch.utils.data.DataLoader(data, batch_size=batch_size,sampler=valid_sampler)


In [105]:
model = Wav2Vec2(n_classes=8)
model = model.cuda()
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.9)

In [112]:
from tqdm import tqdm
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    epoch_loss = 0
    for X, y in tqdm(dataloader,desc=f"Epoch:{t}"):
        #X, y = X.to(device), y.to(device)

        # Compute prediction error
        X = X.float().cuda()
        y = y.squeeze().cuda()
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    print('Training Loss is %.3f'%(epoch_loss/size))



In [113]:
def test(dataloader, model):
    size = len(dataloader.dataset)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            #X, y = X.to(device), y.to(device)
            X = X.float().cuda()
            y = y.squeeze().cuda()
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
epochs = 10
for t in range(epochs):
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model)
print("Done!")